<a href="https://colab.research.google.com/github/BlitzenPrancer/CNN-Construction-Images-Classifier/blob/main/CNN_Construction_Images_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


I am using the ImageDataGenerator class to feed in the training and validation data to the model. This class can also be used to generate augmented data.

In [15]:
train_datagenerator = ImageDataGenerator(rescale=1./255)
test_datagenerator = ImageDataGenerator(rescale=1./255)

train_datagenerator = train_datagenerator.flow_from_directory(
    '/content/drive/MyDrive/code/train',
    target_size=(128,128),
    batch_size=40,
    class_mode='binary')

test_datagenerator = test_datagenerator.flow_from_directory(
    '/content/drive/MyDrive/code/test',
    target_size=(128,128),
    batch_size=10,
    class_mode='binary')

Found 229 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


My model will have 3 Convolution2D layers

In [16]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3),padding='same', activation='relu', input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D((2,2),2),
    
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),     
     
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),   
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [17]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 32768)            

In [18]:
model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(0.001),
             metrics=['accuracy'])

In [19]:
DESIRED_ACCURACY = 0.85

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if((logs.get('accuracy')>DESIRED_ACCURACY) and (logs.get('val_accuracy')>DESIRED_ACCURACY )):
      print("\nReached 85% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [20]:
model.fit_generator(
    train_datagenerator,
    epochs=100,
    validation_data = test_datagenerator,
    callbacks = [callbacks]
    )

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
6/6 [==============================] - 61s 10s/step - loss: 1.5009 - accuracy: 0.4978 - val_loss: 0.6963 - val_accuracy: 0.5000
Epoch 2/100
6/6 [==============================] - 10s 2s/step - loss: 0.6943 - accuracy: 0.5546 - val_loss: 0.6979 - val_accuracy: 0.4750
Epoch 3/100
6/6 [==============================] - 10s 2s/step - loss: 0.6711 - accuracy: 0.5371 - val_loss: 0.6945 - val_accuracy: 0.4750
Epoch 4/100
6/6 [==============================] - 10s 2s/step - loss: 0.6201 - accuracy: 0.6856 - val_loss: 0.7408 - val_accuracy: 0.5000
Epoch 5/100
6/6 [==============================] - 10s 2s/step - loss: 0.5847 - accuracy: 0.7031 - val_loss: 0.7502 - val_accuracy: 0.5000
Epoch 6/100
6/6 [==============================] - 10s 2s/step - loss: 0.5520 - accuracy: 0.7380 - val_loss: 0.6986 - val_accuracy: 0.5750
Epoch 7/100
6/6 [==============================] - 10s 2s/step - loss: 0.4906 - accuracy: 0.7686 - val_loss: 0.7296 - val_accuracy: 0.6000
Epoch 8/100
6/6 [=========

In [21]:
model.save('mymodel.h5')

In [22]:
path = '/content/drive/MyDrive/code/test/unfinished/1.jpg'
img = image.load_img(path, target_size=(128, 128))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images)
print(classes[0])
if classes[0]<0.5:
    print("The Construction of the Building has been Completed")
else:
    print("Building work of the Construction is in Progress.........")

[1.]
Building work of the Construction is in Progress.........
